In [1]:
import numpy as np
import scipy.linalg as la
import pandas as pd

In [111]:
#Produção total
V = pd.read_excel('./dados/Matriz_de_Insumo_Produto_2015_Nivel_67.xls',sheet_name='01',skiprows=5,skipfooter=5,usecols=[*range(7,74)],header=None)
V = np.transpose(V)
#Impostos
Imp = pd.read_excel('./dados/Matriz_de_Insumo_Produto_2015_Nivel_67.xls',sheet_name='01',skiprows=5,skipfooter=5,usecols=[5],header=None)
#Consumo intermediário produção nacional
Un = pd.read_excel('./dados/Matriz_de_Insumo_Produto_2015_Nivel_67.xls',sheet_name='03',skiprows=5,skipfooter=4,usecols=[*range(3,70)],header=None)
#Demanda Final Nacional
Fn = pd.read_excel('./dados/Matriz_de_Insumo_Produto_2015_Nivel_67.xls',sheet_name='03',skiprows=5,skipfooter=4,usecols=[*range(71,77)],header=None)

#Consumo intermediário produção importados
Um = pd.read_excel('./dados/Matriz_de_Insumo_Produto_2015_Nivel_67.xls',sheet_name='04',skiprows=5,skipfooter=4,usecols=[*range(3,70)],header=None)
#Demanda Final Nacional importados
Fm = pd.read_excel('./dados/Matriz_de_Insumo_Produto_2015_Nivel_67.xls',sheet_name='04',skiprows=5,skipfooter=4,usecols=[*range(71,77)],header=None)


# Extrair informações da TRU


In [152]:
#Dados da TRU
planilha = pd.read_excel('./dados/nivel_68_2010_2021_xls/68_tab2_2015.xls',sheet_name='VA',skiprows=5,skipfooter=4,usecols=[*range(1,69)],header=None) # planilha VA
VAB = planilha.iloc[0,:].to_numpy()#Valor adicionado bruto
W = planilha.loc[1,:].to_numpy()#Remuneracoes
RMB = planilha.loc[8,:].to_numpy()#Rendimento misto bruto
Out_Imp = planilha.loc[10,:].to_numpy()# outros impostos e subsidios
Sub = planilha.loc[11,:].to_numpy()# outros impostos e subsidios
Imp_ou_sub = Out_Imp + Sub
o = planilha.loc[13,:]# ocupações

#Matriz para agregação de 68 atividades em 67 da TRU
agg_68_67 = np.concatenate((np.eye(67,41),np.concatenate((np.zeros((40,27)),np.eye(27,27)))),axis=1)
#Transformação nas matrizes para 67 atividades
VAB = np.matmul(agg_68_67,VAB.transpose())
W = np.matmul(agg_68_67,W.transpose())
RMB = np.matmul(agg_68_67,RMB.transpose())
Out_Imp = np.matmul(agg_68_67,Out_Imp.transpose())
Sub = np.matmul(agg_68_67,Sub.transpose())
Imp_ou_sub = np.matmul(agg_68_67,Imp_ou_sub.transpose())
o = np.matmul(agg_68_67,o.transpose())




# Trabalhadar com dados da MIP

In [157]:
g = np.matmul(V,np.ones(127)) ## Produção total por atividade
q = np.transpose(np.matmul(np.transpose(np.ones(67)),V)) ## Produção total por produto
D = np.matmul(V,np.linalg.inv(np.diag(q))) # market share

Bn = np.matmul(Un,np.linalg.inv(np.diag(g))) # consumo intermediário nacional
A = np.matmul(D,Bn) # coeficientes técnicos intersetoriais nacionais

fn = np.matmul(Fn,np.transpose(np.ones(6))) # soma da demanda final
e = np.matmul(D,fn) #Demanda final ( nacional )

Bm = np.matmul(Um,np.linalg.inv(np.diag(g))) # consumo intermediário importado

A_plus = np.matmul(D,Bn+Bm) # coeficientes técnicos ampliada ( contém insumos nacionais e importados )
L = np.linalg.inv(np.eye(A.shape[0])-A) # leontief



In [160]:
l=np.matmul(np.linalg.inv(np.diag(g)),o)#Postos de trabalho por unidade (Milhão) de produção 


In [114]:
Imp.head()

,5
0,204
1,104
2,140
3,1263
4,387


In [188]:
arr = np.zeros(67)
arr[0]=1

In [189]:
np.matmul(np.ones(67),np.matmul(np.matmul(np.matmul(D,Bm),L),arr))

0.15079287164703803

In [170]:
np.matmul(np.matmul(y.transpose(),L),arr)

0.6305090748500009

In [171]:
np.matmul(np.matmul(y.transpose()+tp.transpose(),L),arr)

5    0.977581
dtype: float64

In [173]:
ones(1,67)*D*Bm*L*[1;zeros(66,1)]

5    5995787.0
dtype: float64

In [120]:
w = np.matmul(np.linalg.inv(np.diag(g)),W) # participação de remuneracoes pela produção total, para cada atividade
y=np.matmul(np.linalg.inv(np.diag(g)),VAB) # participação de valor agregado pela produção total, para cada atividade
tp=np.matmul(np.matmul(np.linalg.inv(np.diag(g)),D),Imp)#Divisão de impostos pelo market share de cada produto

to=np.matmul(np.matmul(np.diag(y),np.linalg.inv(np.diag(VAB))),Imp_ou_sub)#Outros impostos sobre a produção

tn = tp+to #Total de impostos sobre produção nacional



In [147]:
massa_salarial = np.matmul(w.transpose(),np.matmul(L,e))

In [148]:
pib = np.matmul(np.matmul(y.transpose()+tp.transpose(),L),e.to_numpy())
Valor_Agregado = np.matmul(np.matmul(y.transpose(),L),e)

In [150]:
massa_salarial/Valor_Agregado

0.5182751729623762

In [92]:
#Massa salarial
y


5155601.0

# Teste de derivação da MIP a partir da TRU ( Andamento )

In [12]:
ConsumoIntermediario = pd.read_excel('./dados/nivel_68_2010_2021_xls/68_tab2_2015.xls',sheet_name='VA',skiprows=5,skipfooter=4,usecols=[*range(1,69)],header=None) # planilha VA
Demanda = pd.read_excel('./dados/nivel_68_2010_2021_xls/68_tab2_2015.xls',sheet_name='VA',skiprows=5,skipfooter=4,usecols=[*range(1,69)],header=None) # planilha VA


In [6]:
Q = np.array([[280,120],[12,8]])
q = np.array([575,20])

print(Q)
print(q)

[[280 120]
 [ 12   8]]
[575  20]


In [14]:
A= np.matmul(Q,np.linalg.inv(np.diag(q)))

In [19]:
eigvector,eigvalues = la.eig(np.matrix.transpose(A))

In [38]:
lambdas = max(eigvector).real
pos_vec = np.argmax(eigvector)

In [39]:
max_eigvectors = eigvalues[:,pos_vec].reshape(2,1)

In [40]:
r = (1-lambdas)/lambdas

In [37]:
lambdas.real

0.8

In [45]:
max_eigvectors/max_eigvectors[0]

array([[ 1.],
       [15.]])

## Importação de dados reais

In [1]:
import pandas as pd


Undefined variable: import
